<a href="https://colab.research.google.com/github/Yidunchibubao/Harmfulness-of-Ambiguous-Vectors/blob/main/prepdata_procedure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!unzip superwikt-fr.zip -d superwikt-fr

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('superwikt-fr/superwikt-fr/2/SuperWikt-fr-coling2025/SuperWikt-fr-dbnary-dump-20230320.tsv', sep='\t')
df.head(20)

In [ ]:
with open("superwikt-fr/superwikt-fr/2/SuperWikt-fr-coling2025/README.txt", "r", encoding="utf-8") as f:
    print(f.read())

In [ ]:
with open("superwikt-fr/superwikt-fr/2/SuperWikt-fr-coling2025/supersense_list.txt", "r", encoding="utf-8") as files_lists:
   for line in files_lists:
      print(line.strip())

In [ ]:
lemma_to_supersense = {}
lemma_to_hypersense = {}

for _, row in df.iterrows():
    lemma = row["lemma"]
    ss = row["supersense"]
    hs = row["hypersense"]

    if lemma not in lemma_to_supersense:
        lemma_to_supersense[lemma] = set()
    if pd.notna(ss):
        lemma_to_supersense[lemma].add(ss)

    if lemma not in lemma_to_hypersense:
        lemma_to_hypersense[lemma] = set()
    if pd.notna(hs):
        lemma_to_hypersense[lemma].add(hs)

lemma_to_supersense = {k: list(v) for k, v in lemma_to_supersense.items()}
lemma_to_hypersense = {k: list(v) for k, v in lemma_to_hypersense.items()}

In [ ]:
print(lemma_to_supersense.get("laveur"))
print(lemma_to_hypersense.get("laveur"))

In [ ]:
list(lemma_to_supersense.items())[:20]

In [ ]:
list(lemma_to_hypersense.items())[:20]

In [ ]:
supersense_set = set(df["supersense"].dropna().unique())
hypersense_set = set(df["hypersense"].dropna().unique())

In [ ]:
print("supersense_set", supersense_set)
print("hypersense_set", hypersense_set)

In [ ]:
print(len(supersense_set))

In [ ]:
print(len(hypersense_set))

In [ ]:
ordinateur = df[df["lemma"] == "ordinateur"]

In [ ]:
leur = df[df["lemma"] == "leur"]
print(leur)

In [ ]:
print(ordinateur)

In [ ]:
ambiguous_sslemmas = []
ambiguous_hslemmas = []
for lemma, supersenses in lemma_to_supersense.items():
    if len(supersenses) > 1:
        ambiguous_sslemmas.append(lemma)

for lemma, hypersenses in lemma_to_hypersense.items():
    if len(hypersenses) > 1:
        ambiguous_hslemmas.append(lemma)

In [ ]:
print("lemmas with more than one supersense")
print(ambiguous_sslemmas)
print(len(ambiguous_sslemmas))

In [ ]:
print("lemmas with more than one hypersense")
print(ambiguous_hslemmas)
print(len(ambiguous_hslemmas))

In [ ]:
!wget https://zenodo.org/records/5975226/files/lemma-A-pos-bow.txt?download=1 -O lemma-A-pos-bow.txt

In [ ]:
import numpy as np

words_bow =[]
words2embeddings_bow = []

with open("lemma-A-pos-bow.txt", "r", encoding="utf-8") as fs:
    header = fs.readline().strip()
    print(header)
    vocab_size, vector_dim = map(int, header.split())

    for line in fs:
        line = line.strip()
        if not line:
            continue
        parts = line.split()
        word = parts[0]
        vector = [float(x) for x in parts[1:]]
        words_bow.append(word)
        words2embeddings_bow.append(vector)

print(len(words_bow))
print(len(words2embeddings_bow))

In [ ]:
embeddings_array_bow = np.array(words2embeddings_bow, dtype=float)
print(embeddings_array_bow.shape)

In [ ]:
print(words_bow.index("laveur_nom"))
print(embeddings_array_bow[28861])

In [ ]:
!wget "https://zenodo.org/records/5975226/files/lemma-A-pos.txt?download=1" -O lemma-A-pos.txt

In [ ]:
words = []
words2embeddings = []

with open("lemma-A-pos.txt", "r", encoding="utf-8") as datas:
  header = datas.readline().strip()
  print(header)
  vocab_size, vocab_dim = map(int, header.split())

  for data in datas:
        data = data.strip()
        if not data:
            continue
        parts = data.split()
        word = parts[0]
        vector = [float(x) for x in parts[1:]]
        words.append(word)
        words2embeddings.append(vector)


In [ ]:
embeddings_array = np.array(words2embeddings, dtype=float)
print(embeddings_array.shape)

In [ ]:
print(words.index("laveur_nom"))

In [ ]:
print(embeddings_array[28861])

In [ ]:
import pickle

prep_data = {
    'lemma_to_supersense': lemma_to_supersense, # dict of lemmas to supersenses
    'lemma_to_hypersense': lemma_to_hypersense, # dict of lemmas to hypersenses
    'supersense_set': supersense_set, # set: all supersenses
    'hypersense_set': hypersense_set, # set: all hypersenses
    'words': words, # list of str: words to embeddings
    'embeddings': embeddings_array, # numpy array of list of list of floats: emdeddings to words
    'words_bow': words_bow, # list of str: words to embeddings
    'embeddings_bow': embeddings_array_bow, # numpy array of list of list of floats: emdeddings to words
    'ambiguous_sslemmas': ambiguous_sslemmas, # list of str
    'ambiguous_hslemmas': ambiguous_hslemmas  # list of str
}

with open('prep_data.pkl', 'wb') as f:
    pickle.dump(prep_data, f)

# one can also check these by using directly the .keys() method after the unpackage of this document

In [ ]:
files.download('prep_data.pkl')